# Airline Delayes and cancellation 

## Introduction
In this project, we would like to make airline delay predictions for the airport JFK in New York. The aim is to find relevant variables we can use to build models for predictions of departure delay in JFK airport. We will use two datasets for the analysis. The first dataset contains information of airline delays and cancellation and the second dataset contains information of weather.

The airline delay dataset consists of more than 6 million flight observations and 28 variables of which some are technical data on airlines, airports, flight numbers, and the rest are time-related. The dataset is from the year 2010.

The weather dataset consists of hourly weather observations for "Temperature" and "Windspeed", for the year 2010.

We will explore the airline delay dataset in order to find relevant variables we can use for predictions of departure delay. Then we will use the variables to build Baysesian linear regression models in Pyro. 